In [2]:

import numpy as np



%load_ext autoreload

import time
import json
import os
import numpy as np
import flwr as fl
import pickle
from math import floor



from logging import INFO, DEBUG
from flwr.common.logger import log



from src.data.dataset_info import datasets
#data_root = "./datasets/partitions/"

In [3]:
NUM_CLIENTS = 10
BATCH_SIZE = 128
folder_path = "datasets/unified/"
clients_paths = [
    folder_path + "client_0.parquet",
    folder_path + "client_1.parquet",
    folder_path + "client_2.parquet",
    folder_path + "client_3.parquet",
    folder_path + "client_4.parquet",
    folder_path + "client_5.parquet",
    folder_path + "client_6.parquet",
    folder_path + "client_7.parquet",
    folder_path + "client_8.parquet",
    folder_path + "client_9.parquet",
]
multi_class = True
dataset = datasets[0]


In [4]:
# specifying the number of classes
classes_set = {"benign", "attack"}
labels_names = {0: "benign", 1: "attack"}
num_classes = 2

if multi_class:
    import json
    with open("label_mappings.json", 'r') as f:
        data = json.load(f)
        labels_names = {int(k): v for k, v in data["labels_names"].items()}
        classes_set = set(data["classes"])
        num_classes = len(classes_set)

labels_names = {int(k): v for k, v in labels_names.items()}  # Ensure keys are int

print(f"==>> classes_set: {classes_set}")
print(f"==>> num_classes: {num_classes}")
print(f"==>> labels_names: {labels_names}")

==>> classes_set: {'PortScan', 'Infiltration', 'Bot', 'DoS slowloris', 'DoS Hulk', 'DoS Slowhttptest', 'SSH-Patator', 'FTP-Patator', 'BENIGN', 'Web Attack � Brute Force', 'DDoS', 'Heartbleed', 'DoS GoldenEye', 'Web Attack � Sql Injection', 'Web Attack � XSS'}
==>> num_classes: 15
==>> labels_names: {0: 'BENIGN', 1: 'Bot', 2: 'DDoS', 3: 'DoS GoldenEye', 4: 'DoS Hulk', 5: 'DoS Slowhttptest', 6: 'DoS slowloris', 7: 'FTP-Patator', 8: 'Heartbleed', 9: 'Infiltration', 10: 'PortScan', 11: 'SSH-Patator', 12: 'Web Attack � Brute Force', 13: 'Web Attack � Sql Injection', 14: 'Web Attack � XSS'}


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split


def read_clients( clients_paths, label_col, class_col, class_num_col, drop_columns, weak_columns):
    test = pd.read_parquet("datasets/unified/test.parquet")

    if multi_class:
        test[label_col] = test[class_num_col]


    test_by_class = {}
    classes = test[class_col].unique()
    for class_value in classes:
        test_class = test[test[class_col] == class_value].copy()
        test_class.drop(drop_columns, axis=1, inplace=True)
        test_class.drop(weak_columns, axis=1, inplace=True)
        test_class.reset_index(drop=True, inplace=True)

        test_class_labels = test_class[label_col].to_numpy()
        test_class = test_class.drop([label_col], axis=1).to_numpy()

        test_by_class[class_value] = (test_class, test_class_labels)

    test.drop(drop_columns, axis=1, inplace=True)
    test.drop(weak_columns, axis=1, inplace=True)
    test.reset_index(drop=True, inplace=True)

    test_labels = test[label_col].to_numpy()
    test = test.drop([label_col], axis=1).to_numpy()
    input_dim = test.shape[1]

    client_data = []
    for client_path in clients_paths:
        client_data.append(pd.read_parquet(client_path))

    for i in range(len(client_data)):

        cdata = client_data[i]

        if multi_class:
            cdata[label_col] = cdata[class_num_col]
       

        cdata.drop(drop_columns, axis=1, inplace=True)
        cdata.drop(weak_columns, axis=1, inplace=True)
        cdata.reset_index(drop=True, inplace=True)

        # Split into train, validation, and test sets
        c_train, c_test = train_test_split(cdata, test_size=0.1)

        # Split c_train further into c_train and c_val
        c_train, c_val = train_test_split(c_train, test_size=0.2)

        # Extract labels and features for train, validation, and test
        y_train = c_train[label_col].to_numpy()
        x_train = c_train.drop([label_col], axis=1).to_numpy()

        y_val = c_val[label_col].to_numpy()
        x_val = c_val.drop([label_col], axis=1).to_numpy()

        y_test = c_test[label_col].to_numpy()
        x_test = c_test.drop([label_col], axis=1).to_numpy()

        # Store in client_data: (x_train, y_train, x_val, y_val, x_test, y_test)
        client_data[i] = (x_train, y_train, x_val, y_val, x_test, y_test)

    return client_data, test, test_labels, test_by_class, input_dim

# Model (don't forget to use class weighting cuz u didn't balance dataset)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

def create_nids_model(input_shape, num_classes):
    model = Sequential(name='NIDS_CNN')
    
    # Feature extraction block 1
    model.add(Conv1D(64, kernel_size=3, activation='relu', 
                     input_shape=input_shape, padding='same',
                     kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    
    # Feature extraction block 2
    model.add(Conv1D(128, kernel_size=3, activation='relu', 
                     padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))
    
    # Feature extraction block 3
    model.add(Conv1D(256, kernel_size=2, activation='relu', 
                     padding='same', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    
    # Classification block
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

# Example usage:
# model = create_nids_model(input_shape=(41, 1), num_classes=5)
# model.summary()

In [7]:
model = create_nids_model(input_shape=(41, 1), num_classes=num_classes)
model.summary()

c:\Users\karayassamine\FL-GDLC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "NIDS_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 41, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 41, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 20, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 10, 256)        │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,335 (1.74 MB)

 Trainable params: 454,927 (1.74 MB)

 Non-trainable params: 1,408 (5.50 KB)

Compute class weights (Client-Level Balancing)

In [8]:
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

def compute_client_weights_multiclass(y_local):
    
    # Calculate class weights
    classes = np.unique(y_local)
    weights = compute_class_weight('balanced', 
                                 classes=classes, 
                                 y=y_local)
    
    # Create weight dictionary
    weight_dict = dict(zip(classes, weights))
    
    # Calculate and print class distribution
    class_counts = {cls: np.sum(y_local == cls) for cls in classes}
    total = len(y_local)
    
    print("\nClass Distribution:")
    for cls, count in class_counts.items():
        print(f"  Class {cls}: {count} samples ({count/total:.2%}) | Weight: {weight_dict[cls]:.4f}")
    
    # Print weight dictionary for reference
    print("\nWeight Dictionary:", weight_dict)
    
    return weight_dict

In [9]:
results_final = {}
results_final["baseline"] = {}
results_final["baseline"]["accuracy"] = {}
results_final["baseline"]["f1s"] = {}


results_final["centralities"] = {}
results_final["centralities"]["accuracy"] = {}
results_final["centralities"]["f1s"] = {}
results_final



{'baseline': {'accuracy': {}, 'f1s': {}},
 'centralities': {'accuracy': {}, 'f1s': {}}}

FL Process

In [10]:
import flwr as fl

class FLClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val, x_test, y_test, input_dim):

        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val  
        self.x_test, self.y_test = x_test, y_test
        self.input_dim = input_dim
    def get_parameters(self):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)
        
    def fit(self, parameters):

        self.model = self.model
        self.set_parameters(parameters)

        history = self.model.fit(self.x_train,self.y_train,
                                validation_data=(self.x_val, self.y_val),  
                                epochs= 3,
                                batch_size=128, 
                                class_weight=compute_class_weight(self.y_train),
                                verbose=0)

        return self.get_parameters(), len(self.x_train), {k: v[-1] for k, v in history.history.items()}


    def evaluate(self, parameters):
        self.set_parameters(parameters)
        loss, accuracy = self.model.evaluate(self.x_val, self.y_val, 2, verbose=0)
        return loss, len(self.x_val), {"accuracy": accuracy}

In [11]:
from sklearn.metrics import accuracy_score

def get_evaluate_fn(x_test_sever, y_test_server, results, test_by_class):

    def evaluate_fn(server_round: int, parameters):
        eval_model = model
        eval_model.set_weights(parameters)

        
        #logdir = "logs/scalars/{}/{}/server".format(dtime, simulation_name) 
        # logdir = "logs/scalars/client{}_".format(config["cid"]) + datetime.now().strftime("%Y%m%d-%H%M%S")
        #tensorboard_callback = TensorBoard(log_dir=logdir)

        test_loss, test_acc = eval_model.evaluate(x_test_sever, y_test_server,
                                                  batch_size = 128,
                                                  )
        
        
        y_pred = eval_model.predict(x_test_sever, batch_size = 128)
        
        if multi_class:
            y_pred = np.argmax(y_pred, axis=1)
            scores =accuracy_score(y_test_server, y_pred)
        else:
            y_pred = np.transpose(y_pred)[0]
            y_pred = list(
                map(lambda x: 0 if x < 0.5 else 1, y_pred))
            scores = accuracy_score(y_test_server, y_pred)
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        
        results["scores"]["accuracy"][server_round] = test_acc
        results["scores"]["f1s"][server_round] = scores["f1s"]
        results["scores"]["server"][server_round] = scores
        
        results_final[simulation_name]["accuracy"][server_round] = scores["accuracy"]
        results_final[simulation_name]["f1s"][server_round] = scores["f1s"]
        
        if not multi_class:
            for k in test_by_class.keys():
                y_pred_class = eval_model.predict(test_by_class[k][0], batch_size = 128, verbose = 0)
                y_pred_class = np.transpose(y_pred_class)[0]
                y_pred_class = list(map(lambda x: 0 if x < 0.5 else 1, y_pred_class))
                scores_class = accuracy_score(test_by_class[k][1], y_pred_class)
                results["scores"]["test_by_class"]["accuracy"][k][server_round] = scores_class["accuracy"]
                results["scores"]["test_by_class"]["f1s"][k][server_round] = scores_class["f1s"]
                
        log(INFO, f"==>> scores: {scores}")
        
        
        return test_loss, {"accuracy": test_acc, "f1s": scores["f1s"], "FPR": scores["FPR"], "FNR": scores["FNR"]}

    return evaluate_fn

In [12]:
def generate_client_fn(data, input_dim):
    def client_fn(cid: str):
        i = int(cid)
        return FLClient(model,data[i][0],  # x_train
            data[i][1],  # y_train
            data[i][2],  # x_val
            data[i][3],  # y_val
            data[i][4],  # x_test
            data[i][5],   # y_test
            input_dim).to_client()

    return client_fn

In [13]:
def get_on_fit_config():

    def fit_config_fn(server_round: int):
        return {
            "lr": 0.001,
            "local_epochs": 1,
            "batch_size": 2,
        }

    return fit_config_fn

# BASELINE

In [14]:
client_data, test, test_labels, test_by_class, input_dim = read_clients(clients_paths,dataset.label_col, dataset.class_col, dataset.class_num_col, dataset.drop_columns, dataset.weak_columns)

In [15]:
results = {}  # a dictionary that will contain all the options and results of models
# add all options to the results dictionary, to know what options selected for obtained results
results["configuration"] = "2dt - baseline"
#results["dtime"] = dtime
results["multi_class"] = multi_class
#results["learning_rate"] = learning_rate
results["dataset_name"] = dataset.name
results["num_classes"] = num_classes
results["labels_names"] = labels_names
results["input_dim"] = input_dim

results["scores"] = {}
results["scores"]["server"] = {}
results["scores"]["clients"] = {}
results["scores"]["accuracy"] = {}
results["scores"]["f1s"] = {}

if not multi_class:
    results["scores"]["test_by_class"] = {}
    results["scores"]["test_by_class"]["accuracy"] = {}
    results["scores"]["test_by_class"]["f1s"] = {}
    for k in test_by_class.keys():
        results["scores"]["test_by_class"]["length"] = len(test_by_class[k][0])
        results["scores"]["test_by_class"]["accuracy"][k] = {}   
        results["scores"]["test_by_class"]["f1s"][k] = {}    
        
results

{'configuration': '2dt - baseline',
 'multi_class': True,
 'dataset_name': 'client_0',
 'num_classes': 15,
 'labels_names': {0: 'BENIGN',
  1: 'Bot',
  2: 'DDoS',
  3: 'DoS GoldenEye',
  4: 'DoS Hulk',
  5: 'DoS Slowhttptest',
  6: 'DoS slowloris',
  7: 'FTP-Patator',
  8: 'Heartbleed',
  9: 'Infiltration',
  10: 'PortScan',
  11: 'SSH-Patator',
  12: 'Web Attack � Brute Force',
  13: 'Web Attack � Sql Injection',
  14: 'Web Attack � XSS'},
 'input_dim': 46,
 'scores': {'server': {}, 'clients': {}, 'accuracy': {}, 'f1s': {}}}

In [16]:

strategy=fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # in simulation, since all clients are available at all times, we can just use `min_fit_clients` to control exactly how many clients we want to involve during fit
    min_fit_clients=len(client_data),  # number of clients to sample for fit()
    fraction_evaluate=0.0,  # similar to fraction_fit, we don't need to use this argument.
    min_evaluate_clients=0,  # number of clients to sample for evaluate()
    min_available_clients=len(client_data),
    evaluate_fn=get_evaluate_fn(test, test_labels, results, test_by_class),
    
)

In [17]:
from flwr.server import ServerApp, ServerAppComponents, ServerConfig

# You need to define the 'strategy' variable before this code
# For example:
# from flwr.server.strategy import FedAvg
# strategy = FedAvg(...)

def generate_server_fn():
    def server_fn(config):  # Accept one argument, even if not used
        return ServerAppComponents(
            strategy=strategy,
            config=ServerConfig(num_rounds=5)
        )

server_app = ServerApp(server_fn=generate_server_fn())



In [18]:
from flwr.client import ClientApp
client_app = ClientApp(client_fn = generate_client_fn(client_data, input_dim))


            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


In [19]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
from math import floor
import multiprocessing

backend_config = {
    "client_resources": {
        "num_cpus": floor(multiprocessing.cpu_count() / len(client_data)),
        # "num_cpus": 1,
        "num_gpus": 0.0,
    }
}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [20]:
from flwr.simulation import run_simulation

# Run simulation
run_simulation(
    server_app = server_app,
    client_app = client_app,
    num_supernodes = NUM_CLIENTS,
    backend_config = backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=1, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
c:\Users\karayassamine\FL-GDLC\.venv\Lib\site-packages\keras\src\backend\tensorflow\core.py:171: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  return np.array(x)
(pid=24352) Stack (most recent call first):
(pid=24352)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed
(pid=24352)   File "<frozen importlib._bootstrap_external>", line 1233 in create_module
(pid=24352)   File "<frozen importlib._bootstrap>", line 573 in module_from_spec
(pid=24352)   File "<frozen importlib._bootstrap>", line 676 in _load_unlocked
(pid=24352)   File "<frozen importlib._bootstrap>", line 1147 in _find_and_load_unlocked
(pid=24352)   File "<frozen importlib._bootstrap>", line 1176 in _f

RuntimeError: Exception in ServerApp thread

# FL-with-centralities